In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 

In [5]:
base_df = pd.read_csv("../data/output_files/final_base_df.csv")

In [6]:
varient_list = list(set(base_df["varient"]))

# Chopped analysis

In [7]:
varient_chopped_counter = {}
for varient in varient_list:
    chopped_counter = len(base_df[(base_df["varient"] == varient )&(base_df["Chopped"] == True)])
    not_chopped_counter = len(base_df[(base_df["varient"] == varient) & (base_df["Chopped"] == False)])
    varient_chopped_counter[varient] = {"chopped":chopped_counter, "not chopped":not_chopped_counter}

print(varient_chopped_counter)

{'AY.4.2': {'chopped': 1165, 'not chopped': 2627}, 'Omicron BA.1': {'chopped': 1165, 'not chopped': 2627}, 'C.1.2': {'chopped': 1162, 'not chopped': 2630}, 'Gamma P.1': {'chopped': 1165, 'not chopped': 2627}, 'Eta B.1.525': {'chopped': 1162, 'not chopped': 2630}, 'Lambda C.37': {'chopped': 1159, 'not chopped': 2633}, 'Kappa B.1.617.1': {'chopped': 1165, 'not chopped': 2627}, 'Delta B.1.617.2': {'chopped': 1165, 'not chopped': 2627}, 'Omicron BA.2': {'chopped': 1162, 'not chopped': 2630}, 'Beta B.1.351': {'chopped': 1162, 'not chopped': 2630}, 'Alpha B.1.1.7': {'chopped': 1159, 'not chopped': 2630}, 'Iota B.1.526': {'chopped': 1162, 'not chopped': 2630}, 'original': {'chopped': 1160, 'not chopped': 2632}, 'Mu B.1.621': {'chopped': 1162, 'not chopped': 2630}}


In [ ]:
labels = varient_list
chopped_counter = (np.divide(list(chopped_dict.values()), number_of_chooped) * 10000).astype(int) / 100
nchopped_counter = (np.divide(list(nchopped_dict.values()), number_of_unChopped) * 10000).astype(int) / 100

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
fig.set_figheight(9)
fig.set_figwidth(25)
fig.set_dpi(200)
rects1 = ax.bar(x - width/2, chopped_counter, width)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Predicted to be chopped by amino acids')
print(x)
print(labels)
ax.set_xticks(x, labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()
plt.xlabel("amino acids")
plt.ylabel("amount")
plt.savefig("plots/aminoacids-chopped-amount.png", dpi=500, bbox_inches='tight')
plt.show()